# 01/20

%matplotlib inline

In [1]:
description = 'mobilenet_from_face_pretrain_sep_feat_all_data'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
train_df = train_df.append(train_df_2)
del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_20_22:15:25.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
mobilenet_pretrained = tf.keras.models.load_model('../models/pretrained_faces/age_gender_tf2_224_deep-03-0.13-0.97.h5')

In [8]:
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [9]:
hidden_layers = [256]

In [10]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat)

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat)

emotion_feat = mobilenet_output
for size in hidden_layers:
    emotion_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_emotion')(emotion_feat)
emotions_count = len(np.unique(train_df.expression))
outputs_emotion = tf.keras.layers.Dense(emotions_count, activation = 'softmax', name = 'outputs_emotion')(emotion_feat)

In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, outputs_emotion) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [14]:
batch_size = 32
epochs = 16

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)

            loss = valence_loss + arousal_loss + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float(loss)
        total_step += 1
        
        total_process += len(row)

        valence_loss = float(valence_loss)
        arousal_loss = float(arousal_loss)
        emotion_loss = float(emotion_loss)
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 6, "valence_loss": 2.8534553050994873, "arousal_loss": 2.515580654144287, "emotion_loss": 2.5623083114624023, "time_take": 4.612616777420044}
{"emotion_correct": 9, "valence_loss": 1.0691165924072266, "arousal_loss": 0.9013432264328003, "emotion_loss": 2.24477219581604, "time_take": 4.974774599075317}
{"emotion_correct": 10, "valence_loss": 1.5327630043029785, "arousal_loss": 1.3371913433074951, "emotion_loss": 2.0266366004943848, "time_take": 5.297564506530762}
{"emotion_correct": 9, "valence_loss": 1.1270781755447388, "arousal_loss": 0.9618244171142578, "emotion_loss": 1.890458583831787, "time_take": 5.668213367462158}
{"emotion_correct": 16, "valence_loss": 0.8901239037513733, "arousal_loss": 0.6736671924591064, "emotion_loss": 1.5974705219268799, "time_take": 6.147798538208008}
{"emotion_correct": 15, "valence_loss": 0.8923944234848022, "arousal_loss": 0.7646913528442383, "emotion_loss": 1.7276701927185059, "time_take": 6.408210039138794}
{"emotion_correct": 12,

{"emotion_correct": 15, "valence_loss": 1.1764898300170898, "arousal_loss": 1.1205782890319824, "emotion_loss": 1.6115731000900269, "time_take": 20.875985860824585}
{"emotion_correct": 17, "valence_loss": 0.8973680734634399, "arousal_loss": 0.7201790809631348, "emotion_loss": 1.2507548332214355, "time_take": 21.16574001312256}
{"emotion_correct": 11, "valence_loss": 1.0063426494598389, "arousal_loss": 0.8074427843093872, "emotion_loss": 1.7273393869400024, "time_take": 21.41614007949829}
{"emotion_correct": 15, "valence_loss": 1.052952766418457, "arousal_loss": 0.9171739816665649, "emotion_loss": 1.328065037727356, "time_take": 21.781331300735474}
{"emotion_correct": 15, "valence_loss": 0.8614622950553894, "arousal_loss": 0.7409919500350952, "emotion_loss": 1.7439146041870117, "time_take": 22.147660970687866}
{"emotion_correct": 22, "valence_loss": 1.2999953031539917, "arousal_loss": 1.082664132118225, "emotion_loss": 1.3846988677978516, "time_take": 22.45090341567993}
{"emotion_correc